# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

In [7]:
import os
import requests
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI


OLLAMA_BASE_URL = "http://localhost:11434"

In [4]:
# Check if Ollama is running
requests.get(f'{OLLAMA_BASE_URL}').content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [ ]:
# Download llama3.2 to your machine 
!ollama pull llama3.2

In [ ]:
# Create OpenAI client with Ollama
ollama = OpenAI(base_url=f'{OLLAMA_BASE_URL}/v1', api_key='ollama')

In [ ]:
class WebsiteSummarizer:
  
    self.system_prompt =  """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""
    self.user_prompt_prefix =  """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.  
"""

    def __init__(self, url):
        self.url = url
        self.contents = fetch_website_contents(url)
        self.summary = self.summarize()
        self.client = ollama
          
        

    def messages_for(self):
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f'{self.user_prompt_prefix}/n{self.contents}'}
        ]

    def summarize(self):
        response = self.client.chat.completions.create(
            model="llama3.2",
            messages=self.messages_for()
        )
        return response.choices[0].message.content
      
    def display_summary(self):
        display(Markdown(self.summary))




        